In [1]:
import keras
import keras.backend as K
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.callbacks import CSVLogger
import os
import keras.activations
import time
import numpy as np

C:\Users\MaksutovRN\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_classes = 10

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

def standardize_sample_wise(dataset):
    standardized = []
    for s in dataset:
        mean, std = s.mean(), s.std()
        s = (s - mean) / std
        standardized.append(s)
    return np.array(standardized)

x_train = standardize_sample_wise(x_train)
x_test = standardize_sample_wise(x_test)

60000 train samples
10000 test samples


In [5]:
batch_size = 64
epochs = 100
units = 64
experiments = 5
start = 0
activations = ['sigmoid', 'tanh', 'relu', 'linear', 'elu', 'selu', 'softplus', 'softsign', 'hard_sigmoid', 'LeakyReLU', 'ThresholdedReLU']
# activations = ['linear']
optimizers = ['rmsp', 'adam', 'sgd', 'Adagrad', 'Adadelta', 'Adamax', 'Nadam']
# optimizers = ['Nadam']

In [6]:
start_time = time.time()
counter = 0
total_items = len(activations) * len(optimizers) * experiments

for act in activations:
    for opt in optimizers:
        print("Training for activation " + act + " with optimizer " + opt)

        for i in range(experiments):
            
            K.clear_session()
            K.reset_uids()
            
            act_dict = {
              'sigmoid': Activation(keras.activations.sigmoid),
              'tanh': Activation(keras.activations.tanh),
              'relu': Activation(keras.activations.relu),
              'linear': Activation(keras.activations.linear),
              'elu': Activation(keras.activations.elu),
              'softplus': Activation(keras.activations.softplus),
              'softsign': Activation(keras.activations.softsign),
              'hard_sigmoid': Activation(keras.activations.hard_sigmoid),
              'LeakyReLU': keras.layers.advanced_activations.LeakyReLU(),
              'selu': Activation(keras.activations.selu),
              'ThresholdedReLU': keras.layers.advanced_activations.ThresholdedReLU(theta=0.7) 
            }

            opt_dict = {
              'rmsp': keras.optimizers.rmsprop(lr=0.001),
              'adam': keras.optimizers.Adam(),
              'sgd': keras.optimizers.SGD(),
              'Adagrad': keras.optimizers.Adagrad(),
              'Adadelta': keras.optimizers.Adadelta(),
              'Adamax': keras.optimizers.Adamax(),
              'Nadam': keras.optimizers.Nadam()
            }
            
            
            model_name = 'std_sample_wise_' + act + "_" + opt + '_' + str(i + start) + '_' + str(units)
            inputs = Input(shape=(784,))
            x = Dense(units)(inputs)
            x = act_dict[act](x)
            x = Dropout(0.2)(x)
            x = Dense(units)(x)
            x = act_dict[act](x)
            x = Dropout(0.2)(x)
            predictions = Dense(num_classes, activation='softmax')(x)
            model = Model(inputs=inputs, outputs=predictions)
                
            model.compile(loss='categorical_crossentropy',
                              optimizer=opt_dict[opt],
                              metrics=['accuracy'])

            csv_logger = CSVLogger('./training_logs/%s.csv' % (model_name), append=False)
            history = model.fit(x_train, y_train,
                                batch_size=batch_size,
                                epochs=epochs,
                                verbose=0,
                                validation_data=(x_test, y_test), callbacks=[csv_logger])

            score = model.evaluate(x_test, y_test, verbose=0)
            print('Test loss:', score[0])
            print('Test accuracy:', score[1])
            
            t = time.time()
            time_diff = t - start_time
            counter +=1
            rem_items = total_items - counter
            total_time = round((total_items / counter) * time_diff)
            rem_time = round(total_time - time_diff)
            m, s = divmod(rem_time, 60)
            h, m = divmod(m, 60)
            d, h = divmod(h, 24)
            print('Remaining time: %d days %d hours %02d minutes %02d seconds' % (d, h, m, s))

Training for activation sigmoid with optimizer rmsp
Test loss: 0.1313122180364324
Test accuracy: 0.9747
Remaining time: 1 days 22 hours 55 minutes 37 seconds
Test loss: 0.13840241661596664
Test accuracy: 0.9715
Remaining time: 1 days 22 hours 43 minutes 15 seconds
Test loss: 0.1449406822345777
Test accuracy: 0.9721
Remaining time: 1 days 22 hours 46 minutes 14 seconds
Test loss: 0.12986746629321425
Test accuracy: 0.9741
Remaining time: 1 days 22 hours 25 minutes 14 seconds
Test loss: 0.1362492784357848
Test accuracy: 0.9739
Remaining time: 1 days 22 hours 16 minutes 31 seconds
Training for activation sigmoid with optimizer adam
Test loss: 0.10722232386910473
Test accuracy: 0.9737
Remaining time: 1 days 22 hours 26 minutes 08 seconds
Test loss: 0.09725193078457378
Test accuracy: 0.974
Remaining time: 1 days 22 hours 32 minutes 17 seconds
Test loss: 0.11226934607489966
Test accuracy: 0.9721
Remaining time: 1 days 22 hours 39 minutes 02 seconds
Test loss: 0.10881804555787239
Test accuracy

Test loss: 0.20540155947553795
Test accuracy: 0.9756
Remaining time: 1 days 16 hours 01 minutes 37 seconds
Test loss: 0.2050534932864163
Test accuracy: 0.9745
Remaining time: 1 days 15 hours 53 minutes 18 seconds
Test loss: 0.21797184327213545
Test accuracy: 0.9726
Remaining time: 1 days 15 hours 44 minutes 59 seconds
Test loss: 0.18465900365171045
Test accuracy: 0.9737
Remaining time: 1 days 15 hours 36 minutes 20 seconds
Test loss: 0.2293217116548385
Test accuracy: 0.9713
Remaining time: 1 days 15 hours 27 minutes 53 seconds
Training for activation relu with optimizer adam
Test loss: 0.10199434833932028
Test accuracy: 0.9777
Remaining time: 1 days 15 hours 20 minutes 29 seconds
Test loss: 0.11017217671257168
Test accuracy: 0.9755
Remaining time: 1 days 15 hours 13 minutes 14 seconds
Test loss: 0.10557379024901893
Test accuracy: 0.9777
Remaining time: 1 days 15 hours 05 minutes 49 seconds
Test loss: 0.11745565520908022
Test accuracy: 0.9736
Remaining time: 1 days 14 hours 58 minutes 1

Test loss: 0.14423262560209668
Test accuracy: 0.9776
Remaining time: 1 days 6 hours 05 minutes 34 seconds
Test loss: 0.14543442366678436
Test accuracy: 0.977
Remaining time: 1 days 5 hours 55 minutes 35 seconds
Test loss: 0.13217917719035904
Test accuracy: 0.9775
Remaining time: 1 days 5 hours 45 minutes 40 seconds
Test loss: 0.1463984617269066
Test accuracy: 0.9765
Remaining time: 1 days 5 hours 35 minutes 47 seconds
Training for activation elu with optimizer adam
Test loss: 0.09954279666810253
Test accuracy: 0.9798
Remaining time: 1 days 5 hours 26 minutes 18 seconds
Test loss: 0.1065233324275534
Test accuracy: 0.9791
Remaining time: 1 days 5 hours 16 minutes 46 seconds
Test loss: 0.10676839723097874
Test accuracy: 0.9787
Remaining time: 1 days 5 hours 07 minutes 21 seconds
Test loss: 0.10262141669290213
Test accuracy: 0.9783
Remaining time: 1 days 4 hours 57 minutes 56 seconds
Test loss: 0.103594902699456
Test accuracy: 0.9762
Remaining time: 1 days 4 hours 48 minutes 30 seconds
Tra

Test loss: 0.20621516873794035
Test accuracy: 0.9732
Remaining time: 0 days 20 hours 01 minutes 28 seconds
Test loss: 0.21955286777437183
Test accuracy: 0.9739
Remaining time: 0 days 19 hours 53 minutes 44 seconds
Test loss: 0.20189168434609647
Test accuracy: 0.9734
Remaining time: 0 days 19 hours 45 minutes 56 seconds
Training for activation softplus with optimizer adam
Test loss: 0.09932111271879293
Test accuracy: 0.9761
Remaining time: 0 days 19 hours 38 minutes 20 seconds
Test loss: 0.10966774943943383
Test accuracy: 0.9755
Remaining time: 0 days 19 hours 30 minutes 42 seconds
Test loss: 0.1119738796512167
Test accuracy: 0.9763
Remaining time: 0 days 19 hours 23 minutes 08 seconds
Test loss: 0.11806621913653871
Test accuracy: 0.9764
Remaining time: 0 days 19 hours 15 minutes 32 seconds
Test loss: 0.10786769264233517
Test accuracy: 0.9776
Remaining time: 0 days 19 hours 07 minutes 58 seconds
Training for activation softplus with optimizer sgd
Test loss: 0.07961166286394
Test accurac

Test loss: 0.14158211726004666
Test accuracy: 0.9716
Remaining time: 0 days 11 hours 29 minutes 28 seconds
Test loss: 0.14619419084046384
Test accuracy: 0.9718
Remaining time: 0 days 11 hours 22 minutes 28 seconds
Test loss: 0.13333764246947466
Test accuracy: 0.975
Remaining time: 0 days 11 hours 15 minutes 28 seconds
Training for activation hard_sigmoid with optimizer adam
Test loss: 0.11394031126194168
Test accuracy: 0.9738
Remaining time: 0 days 11 hours 08 minutes 31 seconds
Test loss: 0.11848076411775546
Test accuracy: 0.9741
Remaining time: 0 days 11 hours 01 minutes 34 seconds
Test loss: 0.10384857663151342
Test accuracy: 0.9728
Remaining time: 0 days 10 hours 54 minutes 40 seconds
Test loss: 0.10681976256802446
Test accuracy: 0.9742
Remaining time: 0 days 10 hours 47 minutes 46 seconds
Test loss: 0.10527317206492881
Test accuracy: 0.9726
Remaining time: 0 days 10 hours 40 minutes 51 seconds
Training for activation hard_sigmoid with optimizer sgd
Test loss: 0.13224089369624853
T

Test loss: 0.31730014966726283
Test accuracy: 0.9622
Remaining time: 0 days 3 hours 32 minutes 06 seconds
Test loss: 0.27185887693508143
Test accuracy: 0.9635
Remaining time: 0 days 3 hours 25 minutes 24 seconds
Test loss: 0.2503298219133168
Test accuracy: 0.9608
Remaining time: 0 days 3 hours 18 minutes 43 seconds
Training for activation ThresholdedReLU with optimizer adam
Test loss: 0.1418162607667342
Test accuracy: 0.9718
Remaining time: 0 days 3 hours 12 minutes 03 seconds
Test loss: 0.1312107408371754
Test accuracy: 0.9719
Remaining time: 0 days 3 hours 05 minutes 24 seconds
Test loss: 0.13238026727979013
Test accuracy: 0.9725
Remaining time: 0 days 2 hours 58 minutes 44 seconds
Test loss: 0.1403273307371419
Test accuracy: 0.9683
Remaining time: 0 days 2 hours 52 minutes 04 seconds
Test loss: 0.1556444229482091
Test accuracy: 0.9678
Remaining time: 0 days 2 hours 45 minutes 25 seconds
Training for activation ThresholdedReLU with optimizer sgd
Test loss: 0.11438983933194541
Test ac